#Prediction Modeling
In this assignment, you will be training and evaluating a range of models using cross validation.

For this assignment, our goal is to train a model to predict whether students are likely to complete their next math assignment.

## Dataset

This assignment will utilize student data collected from the ASSISTments digital learning platform. The dataset contains data from 18,345 unique students across 37,268 assignments. For each of these, our goal is to predict whether each student will complete the assignment using their past performance and behavior in the system. Several student level features have been aggregated for each student up to the moment that they were given the target assignment, and we want to examine whether this information can be used to predict their future performance (as measured by completion of the next assignment).

**The dataset can be downloaded from Canvas or using this direct link:
[ASSISTments Assignments Dataset](https://drive.google.com/file/d/1oEEKSkDPn8RBM6oa9eNmXYIFBmxqFQBm/view?usp=sharing)**

The dataset used in this assignment has been sampled from a larger publicly-available dataset published in the International Conference on Educational Data Mining:

[Prihar, E., Syed, M., Ostrow, K., Shaw, S., Sales, A., & Heffernan, N. (2022, July). Exploring common trends in online educational experiments. *In Proceedings of the 15th International Conference on Educational Data Mining.*](https://educationaldatamining.org/edm2022/proceedings/2022.EDM-long-papers.3/index.html)

**A description of each column and further context for the data can be found in that paper.**

#Data Loading and Preprocessing
Download the **assistments_control_assignments_with_priors.csv** file from Canvas or the link above. Run the first code cell below to upload the dataset. The second code cell below uses the pandas library to read the file into a Dataframe and displays the number of rows and columns as well as a sample of the loaded data.



In [2]:
from google.colab import files
dataset = files.upload()
filename = list(dataset.keys())[0]
print(f"{filename} has been uploaded")

Saving assistments_control_assignments_with_priors.csv to assistments_control_assignments_with_priors.csv
assistments_control_assignments_with_priors.csv has been uploaded


In [3]:
import pandas as pd

# Load the dataset
df = pd.read_csv(filename)

# Drop rows with missing values
df = df.dropna()

# Print the shape of the dataset
print("\nShape of the dataset (rows, columns):", df.shape)

# Display a sample of the dataset
df



Shape of the dataset (rows, columns): (37268, 34)


,problem_set_id,student_id,assignment_completed,normalized_student_learning,0_student_prior_skill_builders,0_student_prior_problem_sets,0_student_prior_attempted_problems,0_student_prior_completed_problems,student_prior_started_skill_builder_count,student_prior_skill_builder_percent_completed,...,class_prior_skill_builder_percent_completed,class_prior_started_problem_set_count,class_prior_problem_set_percent_completed,class_prior_completed_problem_count,class_prior_median_first_response_time,class_prior_median_time_on_task,class_prior_average_attempt_count,class_prior_average_correctness,teacher_account_age_in_days,opportunity_zone
0,PSAUUKY,120915,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
1,PSAUUKY,120915,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
2,PSAUUKY,307046,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,76,1
3,PSAUUKY,307046,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,76,1
4,PSAUUKY,137672,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37263,PSA25TA,1035779,1,-0.125259,0,1,0,0,8,0.875,...,91.914894,0.000000,0.000000,1878,17.7130,25.9645,1.834398,0.567093,508,1
37264,PSA25TA,1031811,1,1.128823,0,1,0,0,10,1.000,...,91.914894,0.000000,0.000000,1878,17.7130,25.9645,1.834398,0.567093,508,1
37265,PSA25TA,1035614,0,-1.379341,0,1,0,0,10,1.000,...,90.306122,0.000000,0.000000,1435,20.6370,29.8810,1.717770,0.567944,508,1
37266,PSA25TA,1035616,1,1.128823,0,1,0,0,10,1.000,...,90.306122,0.000000,0.000000,1435,20.6370,29.8810,1.717770,0.567944,508,1


##Descriptive Statistics

The code cell below identifies all the potential features that are available for us to use for each student; these represent aggregations of prior performance and other measures within the digital learning platform.

In [4]:
# The list of all potential features we can utilize
# Note: class- and teacher-level variables are omitted for this assignment,
#       but these may be useful in other contexts
all_features = ['0_student_prior_skill_builders',
                      '0_student_prior_problem_sets',
                      '0_student_prior_attempted_problems',
                      '0_student_prior_completed_problems',
                      'student_prior_started_skill_builder_count',
                      'student_prior_skill_builder_percent_completed',
                      'student_prior_started_problem_set_count',
                      'student_prior_problem_set_percent_completed',
                      'student_prior_completed_problem_count',
                      'student_prior_median_first_response_time',
                      'student_prior_median_time_on_task',
                      'student_prior_average_attempt_count',
                      'student_prior_average_correctness',
                      'opportunity_zone']

# The following represents one possible permutation of features
low_corr_features = ['0_student_prior_completed_problems',
                      'student_prior_started_skill_builder_count',
                      'student_prior_skill_builder_percent_completed',
                      'student_prior_problem_set_percent_completed',
                      'student_prior_completed_problem_count',
                      'student_prior_median_first_response_time',
                      'student_prior_average_attempt_count',
                      'student_prior_average_correctness',
                      'opportunity_zone']

dependent_variable = "assignment_completed"
df_features = df[all_features]

# Display the means and standard deviations
stats = {}
stats['Mean'] = df_features.mean(numeric_only=True)
stats['SD'] = df_features.std(numeric_only=True)

feature_statistics = pd.DataFrame(stats)

print(f"The dataset contains a total of {len(df)} rows and {len(df.columns)} columns")
print(f"{len(all_features)} potential feature columns have been identified:")
print(feature_statistics.round(2))
print("==========")
print(f"Dependent Measure: {dependent_variable}")
print(f"{round(df[dependent_variable].mean()*100,2)}% of the {df[dependent_variable].sum()} total assignments were completed")

The dataset contains a total of 37268 rows and 34 columns
14 potential feature columns have been identified:
                                                 Mean       SD
0_student_prior_skill_builders                   0.19     0.40
0_student_prior_problem_sets                     0.24     0.43
0_student_prior_attempted_problems               0.06     0.24
0_student_prior_completed_problems               0.06     0.24
student_prior_started_skill_builder_count        6.45     8.73
student_prior_skill_builder_percent_completed    0.65     0.40
student_prior_started_problem_set_count         18.26    30.63
student_prior_problem_set_percent_completed      0.63     0.42
student_prior_completed_problem_count          175.76   268.13
student_prior_median_first_response_time        97.14  5148.59
student_prior_median_time_on_task              106.91  5148.64
student_prior_average_attempt_count              1.32     0.47
student_prior_average_correctness                0.62     0.23
opportuni

# Challenge Solution
Compare your solution to the code below. Note, we implement a simple K-Fold cross validation here, but in the next part, we will examine a student-level cross validation.

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score
from sklearn.model_selection import KFold
import numpy as np

# Create variables for our dependent variable and the set of selected features
X = df[low_corr_features]
y = df[dependent_variable]

# Prepare a 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Lists to store metrics for each fold
accuracies = []
kappas = []
aucs = []

# Loop through each fold...
for train_index, test_index in kf.split(X):
    # Split data
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Fit and train the logistic regression model with sklearn
    lr_model = LogisticRegression()
    lr_model.fit(X_train, y_train)

    # Predictions using sklearn model
    y_pred_sklearn = lr_model.predict(X_test)
    y_pred_proba_sklearn = lr_model.predict_proba(X_test)[:, 1]

    # Model performance metrics using sklearn predictions
    accuracy = accuracy_score(y_test, y_pred_sklearn)
    kappa = cohen_kappa_score(y_test, y_pred_sklearn)
    auc = roc_auc_score(y_test, y_pred_proba_sklearn)

    accuracies.append(accuracy)
    kappas.append(kappa)
    aucs.append(auc)

# Create a DataFrame to store performance metrics
performance = pd.DataFrame({'Metric': ['Accuracy', 'Kappa', 'AUC'],
                            'Value': [np.mean(accuracies), np.mean(kappas), np.mean(aucs)]})

# Display performance metrics and regression coefficients
print("Performance Metrics using scikit-learn:")
print(performance)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Performance Metrics using scikit-learn:
     Metric     Value
0  Accuracy  0.772513
1     Kappa  0.064293
2       AUC  0.680953


In [6]:
import pandas as pd

# Load the dataset
df = pd.read_csv(filename)

# Drop rows with missing values
df = df.dropna()

# Print the shape of the dataset
print("\nShape of the dataset (rows, columns):", df.shape)

# Display a sample of the dataset
df



Shape of the dataset (rows, columns): (37268, 34)


,problem_set_id,student_id,assignment_completed,normalized_student_learning,0_student_prior_skill_builders,0_student_prior_problem_sets,0_student_prior_attempted_problems,0_student_prior_completed_problems,student_prior_started_skill_builder_count,student_prior_skill_builder_percent_completed,...,class_prior_skill_builder_percent_completed,class_prior_started_problem_set_count,class_prior_problem_set_percent_completed,class_prior_completed_problem_count,class_prior_median_first_response_time,class_prior_median_time_on_task,class_prior_average_attempt_count,class_prior_average_correctness,teacher_account_age_in_days,opportunity_zone
0,PSAUUKY,120915,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
1,PSAUUKY,120915,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
2,PSAUUKY,307046,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,76,1
3,PSAUUKY,307046,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,76,1
4,PSAUUKY,137672,1,0.895579,1,0,0,0,0,0.000,...,0.000000,1942.857143,77.205882,3248,25.5405,33.1140,1.314039,0.666436,75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37263,PSA25TA,1035779,1,-0.125259,0,1,0,0,8,0.875,...,91.914894,0.000000,0.000000,1878,17.7130,25.9645,1.834398,0.567093,508,1
37264,PSA25TA,1031811,1,1.128823,0,1,0,0,10,1.000,...,91.914894,0.000000,0.000000,1878,17.7130,25.9645,1.834398,0.567093,508,1
37265,PSA25TA,1035614,0,-1.379341,0,1,0,0,10,1.000,...,90.306122,0.000000,0.000000,1435,20.6370,29.8810,1.717770,0.567944,508,1
37266,PSA25TA,1035616,1,1.128823,0,1,0,0,10,1.000,...,90.306122,0.000000,0.000000,1435,20.6370,29.8810,1.717770,0.567944,508,1


# Student-Level Cross Validation
To apply student-level cross validation, we can use the "GroupKFold" function from sklearn and provide it with the student identifiers from our dataset. It is usually good practice to also randomly shuffle your data before using this function in case your data is ordered in a systematic way.

The code below applies this folding method and displays the number of training and test samples for each fold.

In [7]:
from sklearn.model_selection import GroupKFold
import numpy as np

X_all = df[all_features]
X_selected = df[low_corr_features]
y = df[dependent_variable]

groups = df['student_id'].unique()
np.random.seed(42)  # Set the random seed for reproducibility
np.random.shuffle(groups)  # Shuffle the group identifiers
shuffled_students = df['student_id'].map(dict(zip(groups, np.arange(len(groups)))))

folds = list(GroupKFold(n_splits=10).split(X_all, y, groups=shuffled_students))

for fold_idx, (train_idx, test_idx) in enumerate(folds, 1):
    X_train, X_test = X_all.iloc[train_idx], X_all.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Count samples in each fold
    num_train_samples = len(X_train)
    num_test_samples = len(X_test)
    print(f"Fold {fold_idx}: {num_train_samples} training samples, {num_test_samples} test samples")

Fold 1: 33541 training samples, 3727 test samples
Fold 2: 33541 training samples, 3727 test samples
Fold 3: 33541 training samples, 3727 test samples
Fold 4: 33541 training samples, 3727 test samples
Fold 5: 33541 training samples, 3727 test samples
Fold 6: 33541 training samples, 3727 test samples
Fold 7: 33541 training samples, 3727 test samples
Fold 8: 33541 training samples, 3727 test samples
Fold 9: 33542 training samples, 3726 test samples
Fold 10: 33542 training samples, 3726 test samples


##Logistic Regression

Using the student-level folds from above, we can train and evaluate a Logistic Regression model.

Note: Not all models require us to use low-correlated features. For each model we will need to decide which feature set is most appropriate.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_squared_error

metrics = {
    'AUC': [],
    'Kappa': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1': [],
    'RMSE': [],
}

# Loop through the folded data to train and evaluate our model
for train_idx, test_idx in folds:

    # NOTE: The test set is built from the current fold, while
    # the training set is built from the remaining folds
    # these are stored as sets of indices from the sklearn function

    # X_all contains all potential features
    X_all_train, X_all_test = X_all.iloc[train_idx], X_all.iloc[test_idx]

    # X_selected contains only features with low correlations
    X_selected_train, X_selected_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]

    X_train, X_test = X_selected_train, X_selected_test # or X_all_train, X_all_test
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Fit and train the logistic regression model with sklearn
    lr_model = LogisticRegression(max_iter=1000)
    lr_model.fit(X_train, y_train)

    # Predictions using sklearn model
    y_classification = lr_model.predict(X_test)
    y_probability = lr_model.predict_proba(X_test)[:, 1]

    # Model performance metrics using sklearn predictions
    metrics['AUC'].append(roc_auc_score(y_test, y_probability))
    metrics['Accuracy'].append(accuracy_score(y_test, y_classification))
    metrics['Precision'].append(precision_score(y_test, y_classification))
    metrics['Recall'].append(recall_score(y_test, y_classification))
    metrics['F1'].append(f1_score(y_test, y_classification))
    metrics['Kappa'].append(cohen_kappa_score(y_test, y_classification))
    metrics['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_probability)))

# Display performance metrics and regression coefficients
print("Performance Metrics using scikit-learn:")
for metric in metrics.keys():
    if metrics[metric]:  # Check if metric list is not empty
        print(f"{metric}: {np.mean(metrics[metric]):.3f}")


Performance Metrics using scikit-learn:
AUC: 0.672
Kappa: 0.055
Accuracy: 0.771
Precision: 0.776
Recall: 0.987
F1: 0.869
RMSE: 0.407


## Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_squared_error

metrics = {
    'AUC': [],
    'Kappa': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1': [],
    'RMSE': [],
}

# Loop through the folded data to train and evaluate our model
for train_idx, test_idx in folds:

    # NOTE: The test set is built from the current fold, while
    # the training set is built from the remaining folds
    # these are stored as sets of indices from the sklearn function

    # X_all contains all potential features
    X_all_train, X_all_test = X_all.iloc[train_idx], X_all.iloc[test_idx]

    # X_selected contains only features with low correlations
    X_selected_train, X_selected_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]

    X_train, X_test = X_all_train, X_all_test # or X_selected_train, X_selected_test
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Define hyperparameters
    max_depth = 5  # Maximum depth of the tree
    min_samples_split = 2  # Minimum number of samples required to split an internal node
    min_samples_leaf = 3  # Minimum number of samples required to be at a leaf node

    # Initialize and train the Decision Tree classifier with specified hyperparameters
    dt_model = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
    dt_model.fit(X_train, y_train)

    # Predictions using sklearn model
    y_classification = dt_model.predict(X_test)
    y_probability = dt_model.predict_proba(X_test)[:, 1]

    # Model performance metrics using sklearn predictions
    metrics['AUC'].append(roc_auc_score(y_test, y_probability))
    metrics['Accuracy'].append(accuracy_score(y_test, y_classification))
    metrics['Precision'].append(precision_score(y_test, y_classification))
    metrics['Recall'].append(recall_score(y_test, y_classification))
    metrics['F1'].append(f1_score(y_test, y_classification))
    metrics['Kappa'].append(cohen_kappa_score(y_test, y_classification))
    metrics['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_probability)))

# Display performance metrics and regression coefficients
print("Performance Metrics using scikit-learn:")
for metric in metrics.keys():
    if metrics[metric]:  # Check if metric list is not empty
        print(f"{metric}: {np.mean(metrics[metric]):.3f}")

Performance Metrics using scikit-learn:
AUC: 0.712
Kappa: 0.145
Accuracy: 0.771
Precision: 0.791
Recall: 0.956
F1: 0.865
RMSE: 0.400


## Naive Bayes

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_squared_error

metrics = {
    'AUC': [],
    'Kappa': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1': [],
    'RMSE': [],
}

# Loop through the folded data to train and evaluate our model
for train_idx, test_idx in folds:

    # NOTE: The test set is built from the current fold, while
    # the training set is built from the remaining folds
    # these are stored as sets of indices from the sklearn function

    # X_all contains all potential features
    X_all_train, X_all_test = X_all.iloc[train_idx], X_all.iloc[test_idx]

    # X_selected contains only features with low correlations
    X_selected_train, X_selected_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]

    X_train, X_test = X_selected_train, X_selected_test # or X_all_train, X_all_test
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Initialize and train the Naive Bayes model
    nb_model = GaussianNB()
    nb_model.fit(X_train, y_train)

    # Predictions using sklearn model
    y_classification = nb_model.predict(X_test)
    y_probability = nb_model.predict_proba(X_test)[:, 1]

    # Model performance metrics using sklearn predictions
    metrics['AUC'].append(roc_auc_score(y_test, y_probability))
    metrics['Accuracy'].append(accuracy_score(y_test, y_classification))
    metrics['Precision'].append(precision_score(y_test, y_classification))
    metrics['Recall'].append(recall_score(y_test, y_classification))
    metrics['F1'].append(f1_score(y_test, y_classification))
    metrics['Kappa'].append(cohen_kappa_score(y_test, y_classification))
    metrics['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_probability)))

# Display performance metrics and regression coefficients
print("Performance Metrics using scikit-learn:")
for metric in metrics.keys():
    if metrics[metric]:  # Check if metric list is not empty
        print(f"{metric}: {np.mean(metrics[metric]):.3f}")

Performance Metrics using scikit-learn:
AUC: 0.663
Kappa: 0.049
Accuracy: 0.740
Precision: 0.777
Recall: 0.929
F1: 0.846
RMSE: 0.456


## Support Vector Machine

from sklearn.svm import SVC # sklearn's implementation of SVM
from sklearn.metrics import accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
np.random.seed(42)

metrics = {
    'AUC': [],
    'Kappa': [],
    'Accuracy': [],
    'Precision': [],
    'Recall': [],
    'F1': [],
    'RMSE': [],
}

# Loop through the folded data to train and evaluate our model
for train_idx, test_idx in folds:

    # NOTE: The test set is built from the current fold, while
    # the training set is built from the remaining folds
    # these are stored as sets of indices from the sklearn function

    # NOTE: SVM models can take a long time to train, especially with large datasets
    # for model exploration, it is common to use a random subset of the data to increase the training time
    # For this assignment, let's use 25% of the training set, but feel welcomed to increase this
    subset_fraction = 0.5  # Fraction of the dataset to sample, e.g., 50%
    num_samples = int(len(train_idx) * subset_fraction)

    # Randomly select indices without replacement
    sampled_indices = np.random.choice(train_idx, size=num_samples, replace=False)

    # X_all contains all potential features
    X_all_train, X_all_test = X_all.iloc[sampled_indices], X_all.iloc[test_idx]

    # X_selected contains only features with low correlations
    X_selected_train, X_selected_test = X_selected.iloc[sampled_indices], X_selected.iloc[test_idx]

    X_train, X_test = X_all_train, X_all_test # or X_selected_train, X_selected_test
    y_train, y_test = y.iloc[sampled_indices], y.iloc[test_idx]

    # Define SVM hyperparameters
    # NOTE: Some hyperparameter combinations may drastically increase the training time
    C = 1.0  # Regularization parameter
    kernel = 'linear'  # Type of SVM kernel

    # Initialize and train the SVM with specified hyperparameters
    # It's a good practice to scale your data for SVM models, so we use make_pipeline
    svm_model = make_pipeline(StandardScaler(), SVC(C=C, kernel=kernel, probability=True, cache_size=2048))
    svm_model.fit(X_train, y_train)

    # Predictions using sklearn model
    y_classification = svm_model.predict(X_test)
    y_probability = svm_model.predict_proba(X_test)[:, 1]

    # Model performance metrics using sklearn predictions
    metrics['AUC'].append(roc_auc_score(y_test, y_probability))
    metrics['Accuracy'].append(accuracy_score(y_test, y_classification))
    metrics['Precision'].append(precision_score(y_test, y_classification))
    metrics['Recall'].append(recall_score(y_test, y_classification))
    metrics['F1'].append(f1_score(y_test, y_classification))
    metrics['Kappa'].append(cohen_kappa_score(y_test, y_classification))
    metrics['RMSE'].append(np.sqrt(mean_squared_error(y_test, y_probability)))

# Display performance metrics and regression coefficients
print("Performance Metrics using scikit-learn:")
for metric in metrics.keys():
    if metrics[metric]:  # Check if metric list is not empty
        print(f"{metric}: {np.mean(metrics[metric]):.3f}")